# Introduction
State notebook purpose here

### Get source folder and append to sys directory

In [1]:
from __future__ import print_function
import os
import sys
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)
# Data path example
#pump_data_path = os.path.join(PROJ_ROOT,
#                              "data",
#                              "raw",
#                              "pumps_train_values.csv")

/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program


### Imports
Import libraries and write settings here.

In [5]:
# Data manipulation
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from tensorflow import keras
from tensorflow.keras import layers
import glob
import xarray as xr
from sklearn.preprocessing import MinMaxScaler
import holidays

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython import get_ipython
ipython = get_ipython()
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 1
# Use %aimport module to reload each module

# Extract Data
# %aimport features.extract_features
from features import extract_features
%aimport data.create_input_for_models
from data import create_load_transform_processed_data, create_input_for_models
# Make dataset

# Visualizations
import matplotlib.pyplot as plt

# Analysis/Modeling
Do work here

### Get interim data

In [3]:
_interim_data_path = os.path.join(PROJ_ROOT,
                                  "data",
                                  "interim")
_interim_files = glob.glob(_interim_data_path + '/*.csv')

interim_df = pd.DataFrame()
for file in _interim_files:
    print('Currently processing file \n{}'.format(file))
    interim_df = interim_df.append(pd.read_csv(file, parse_dates=True, index_col=['site_id', 'time'],
                                                     dtype={'CO': np.float64, 'NO2': np.float64, 'PM25': np.float64,
                                                            'AQI_h': np.float64, 'AQI_h_I': np.int, 'site_id': np.int}))
# Site 16 have many inconsistency in data so we remove it
interim_df = interim_df[(interim_df.index.get_level_values(0) != 16)]
# Get only columns we need
interim_df = interim_df[['PM25', 'AQI_h', 'AQI_h_Polutant', 'AQI_h_I',
       'AQI_h_label', 'Continous length']]
# Ho Chi Minh data is on site 49
hanoi_df = interim_df[(interim_df.index.get_level_values(0) != 49)].copy()
hcm_df = interim_df[(interim_df.index.get_level_values(0) == 49)].copy()

Currently processing file 
../data/interim/30.csv
Currently processing file 
../data/interim/9.csv
Currently processing file 
../data/interim/32.csv
Currently processing file 
../data/interim/11.csv
Currently processing file 
../data/interim/40.csv
Currently processing file 
../data/interim/28.csv
Currently processing file 
../data/interim/49.csv
Currently processing file 
../data/interim/46.csv
Currently processing file 
../data/interim/10.csv
Currently processing file 
../data/interim/8.csv
Currently processing file 
../data/interim/25.csv
Currently processing file 
../data/interim/7.csv
Currently processing file 
../data/interim/16.csv
Currently processing file 
../data/interim/42.csv
Currently processing file 
../data/interim/44.csv
Currently processing file 
../data/interim/37.csv
Currently processing file 
../data/interim/1.csv
Currently processing file 
../data/interim/13.csv
Currently processing file 
../data/interim/31.csv
Currently processing file 
../data/interim/26.csv
Curr

In [4]:
hcm_df

PM25  AQI_h AQI_h_Polutant  AQI_h_I AQI_h_label  \
site_id time                                                                   
49      2021-07-01 02:00:00  31.0   31.0           PM25        1        Good   
        2021-07-01 03:00:00  31.0   31.0           PM25        1        Good   
        2021-07-01 04:00:00  31.0   31.0           PM25        1        Good   
        2021-07-01 05:00:00  24.0   24.0           PM25        1        Good   
        2021-07-01 06:00:00  28.0   28.0           PM25        1        Good   
...                           ...    ...            ...      ...         ...   
        2021-07-19 20:00:00  36.0   36.0           PM25        1        Good   
        2021-07-19 21:00:00  32.0   32.0           PM25        1        Good   
        2021-07-19 22:00:00  29.0   29.0           PM25        1        Good   
        2021-07-19 23:00:00  22.0   22.0           PM25        1        Good   
        2021-07-20 00:00:00  15.0   15.0           PM25        1        Good   

                             Continous length  
site_id time                                   
49      2021-07-01 02:00:00             148.0  
        2021-07-01 03:00:00               0.0  
        2021-07-01 04:00:00               0.0  
        2021-07-01 05:00:00               0.0  
        2021-07-01 06:00:00               0.0  
...                                       ...  
        2021-07-19 20:00:00               0.0  
        2021-07-19 21:00:00               0.0  
        2021-07-19 22:00:00               0.0  
        2021-07-19 23:00:00               0.0  
        2021-07-20 00:00:00               0.0  

[439 rows x 6 columns]

### Create input for model from interim data and put in to processed

In [4]:
_model_input_data_path = os.path.join(PROJ_ROOT,
                                   "data",
                                   "model_input")
target_hour = [1, 2, 3, 4, 5]
timesteps = [5]
train, y_train = create_input_for_models.create(hcm_df, timesteps=timesteps, target_hour=target_hour, test_output=True, dev_output=True, output_path=_model_input_data_path, PROJ_ROOT=PROJ_ROOT)

[1, 2, 3, 4, 5]
Feature shape:  (4722, 5, 34)
Label shape:  (4722,)
Data array:
shape of arr:  (4722, 5, 34)
shape of loaded_array:  (4722, 5, 34)
Yes, both the arrays are same
Label array:
shape of arr:  (4722,)
shape of loaded_array:  (4722,)
Yes, both the arrays are same
Feature shape:  (4219, 5, 34)
Label shape:  (4219,)
Data array:
shape of arr:  (4219, 5, 34)
shape of loaded_array:  (4219, 5, 34)
Yes, both the arrays are same
Label array:
shape of arr:  (4219,)
shape of loaded_array:  (4219,)
Yes, both the arrays are same
Feature shape:  (36945, 5, 34)
Label shape:  (36945,)
Data array:
shape of arr:  (36945, 5, 34)
shape of loaded_array:  (36945, 5, 34)
Yes, both the arrays are same
Label array:
shape of arr:  (36945,)
shape of loaded_array:  (36945,)
Yes, both the arrays are same
Feature shape:  (4721, 5, 34)
Label shape:  (4721,)
Data array:
shape of arr:  (4721, 5, 34)
shape of loaded_array:  (4721, 5, 34)
Yes, both the arrays are same
Label array:
shape of arr:  (4721,)
shap

In [8]:
import joblib
_scaler_input_data_path = os.path.join(PROJ_ROOT,
                                   "data",
                                   "model_input",
                                   "final_hcm_05",
                                   "scaler.pkl")
scaler = joblib.load(_scaler_input_data_path)

In [9]:
hcm_df[['AQI_h']] = scaler.transform(hcm_df[['AQI_h']])
hcm_df

PM25     AQI_h AQI_h_Polutant  AQI_h_I  \
site_id time                                                          
49      2021-07-01 02:00:00  31.0 -0.778626           PM25        1   
        2021-07-01 03:00:00  31.0 -0.778626           PM25        1   
        2021-07-01 04:00:00  31.0 -0.778626           PM25        1   
        2021-07-01 05:00:00  24.0 -0.832061           PM25        1   
        2021-07-01 06:00:00  28.0 -0.801527           PM25        1   
...                           ...       ...            ...      ...   
        2021-07-19 20:00:00  36.0 -0.740458           PM25        1   
        2021-07-19 21:00:00  32.0 -0.770992           PM25        1   
        2021-07-19 22:00:00  29.0 -0.793893           PM25        1   
        2021-07-19 23:00:00  22.0 -0.847328           PM25        1   
        2021-07-20 00:00:00  15.0 -0.900763           PM25        1   

                            AQI_h_label  Continous length  
site_id time                                               
49      2021-07-01 02:00:00        Good             148.0  
        2021-07-01 03:00:00        Good               0.0  
        2021-07-01 04:00:00        Good               0.0  
        2021-07-01 05:00:00        Good               0.0  
        2021-07-01 06:00:00        Good               0.0  
...                                 ...               ...  
        2021-07-19 20:00:00        Good               0.0  
        2021-07-19 21:00:00        Good               0.0  
        2021-07-19 22:00:00        Good               0.0  
        2021-07-19 23:00:00        Good               0.0  
        2021-07-20 00:00:00        Good               0.0  

[439 rows x 6 columns]

In [12]:
_model_input_data_path = os.path.join(PROJ_ROOT,
                                   "data",
                                   "model_input",
                                   "aqinow",)
target_hour = [1]
timesteps = [5]
train, y_train = create_input_for_models.create(hcm_df, timesteps=timesteps, target_hour=target_hour, test_output=False, dev_output=False, scale_data=False, output_path=_model_input_data_path, PROJ_ROOT=PROJ_ROOT)

[1]
Feature shape:  (422, 5, 34)
Label shape:  (422,)
Data array:
shape of arr:  (422, 5, 34)
shape of loaded_array:  (422, 5, 34)
Yes, both the arrays are same
Label array:
shape of arr:  (422,)
shape of loaded_array:  (422,)
Yes, both the arrays are same
Input have been created


In [8]:
y_train

array([-0.648855  , -0.7251908 , -0.77862597, ..., -0.8015267 ,
       -0.8167939 , -0.8015267 ], dtype=float32)

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here